# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [3]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
# Get gpt-4o-mini to answer, with streaming

system_prompt="""You are provided with lines of code, and are capable of determining which programming language the code is even if the prompt provided \
does not specify which language it is from. You are an assistant to software engineers that have basic knowledge in programming, and you \
should determine which programming language it is using and what the code is doing. You should respond with examples showing the usage of the code \
that has been provided. 
When giving examples of the code or explaning a line of code, display it so that when displayed in Markdown, it will be displayed in a code block.
"""

def answer_question(question):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


answer_question(question)

The code you've provided is written in Python. Specifically, it uses the `yield from` syntax introduced in Python 3.3, alongside a set comprehension.

### Explanation

1. **Set Comprehension**:
   python
   {book.get("author") for book in books if book.get("author")}
   
   This part of the code creates a set of authors from a list or collection named `books`. Each item in `books` is expected to be a dictionary that may contain an "author" key. The comprehension iterates through each `book`, retrieves the value associated with the "author" key using `book.get("author")`, and adds it to a set if the value is not `None` (or falsy). This ensures that only unique author names are added to the set, as sets do not allow duplicate entries.

2. **Yield from**:
   python
   yield from { ... }
   
   The `yield from` expression is used in a generator function to yield all values from the iterable directly. In this case, it yields each author name from the set created in the previous step. This means that rather than returning the set at once (like with a normal return statement), this code will yield each author one by one when the generator function is called.

### Usage Example

Let's say you have a list of books structured as follows:

python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four", "author": None},
    {"title": "Book Five", "author": "Author C"},
]


You can create a generator function that utilizes the provided code:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
for author in unique_authors(books):
    print(author)


### Output:
This code will produce the following output:


Author C
Author B
Author A


### Summary
- The code creates a generator that yields unique authors of books.
- It filters out any `None` values (i.e., books without an author).
- It's useful for situations where you need to iterate through unique values without storing them all in memory at once.

In [15]:
# Get Llama 3.2 to answer
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

**Code Explanation**

The given code is written in Python. It appears to be using a combination of generator expressions and the `yield from` statement.

Here's a breakdown of what the code does:

* `yield from {book.get("author") for book in books if book.get("author")}` can be simplified as:
```python
for book in books:
    if book.get("author"):
        yield book.get("author")
```
However, the original version using dictionary comprehension and `yield from` is more concise.

This code extracts author names from a list of books (`books`) that contain an "author" key. The resulting generator yields only the author names.

**Why it's used**

The use of `yield from` here is to delegate to another iterable (in this case, another generator expression) and have the current generator remember its state. This allows us to write a more readable and concise code than we would if we were writing multiple nested loops or conditionals.

In this specific example, using `yield from` makes it easie